In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\danis\AppData\Local\Temp\ipykernel_20520\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re


- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [4]:
## Read Data for the Fraudulent Email Kaggle Challenge
data_train = pd.read_csv("../data/kg_train.csv",encoding='latin-1')
data_test = pd.read_csv("../data/kg_test.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data_train = data_train[:1000]
print(data_train.shape)
data_train.fillna("",inplace=True)
data_test.fillna("",inplace=True)
data_train.head(100)


(1000, 2)


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
...,...,...
95,Subject to your satisfaction you will be given...,1
96,Also I should mention the G-20.,0
97,RGVhciBmcmllbmQsIAoKR3JlZXRpbmdzIHRvIHlvdSBpbi...,1
98,cheryl.mills B6Sunday August 2 2009 8:50 AMHRe...,0


### Let's divide the training and test set into two partitions

In [4]:
X = data_train['text']
y = data_train['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)


## Data Preprocessing

In [5]:
import string
import nltk
#nltk.download()

from nltk.corpus import stopwords
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [6]:

def remove_js_css_html_code(text):
    # Define regex for JS/CSS
    regex_JS = '<script\b[^<]*(?:(?!<\/script>)<[^<]*)*<\/script>'
    regex_CSS = '<style\b[^<]*(?:(?!<\/style>)<[^<]*)*<\/style>'
    regex_html = '<!--(.*?)-->'

    # Remove <script> ... </script> (JavaScript code)
    pattern_JS = re.compile('[%s]' % regex_JS) 
    text_no_JS = pattern_JS.sub(u' ', text, re.DOTALL | re.IGNORECASE)
    
    # Remove <style> ... </style> (CSS code)
    pattern_CSS = re.compile('[%s]' % regex_CSS) 
    text_no_JS_no_CSS = pattern_CSS.sub(u' ', text_no_JS, re.DOTALL | re.IGNORECASE)
    
    # Remove HTML comments (<!-- ... -->)
    pattern_html = re.compile('[%s]' % regex_html) 
    text_no_JS_no_CSS_no_html = pattern_html.sub(u' ', text_no_JS_no_CSS, re.DOTALL)

    # Remove remaining HTML tags by repeating the html pattern
    text_no_JS_no_CSS_no_html_no_tags = pattern_html.sub(u' ', text_no_JS_no_CSS_no_html, )  
    
    return text_no_JS_no_CSS_no_html_no_tags

X_train = X_train.apply(remove_js_css_html_code)
X_test = X_test.apply(remove_js_css_html_code)
X_train.head()

<>:3: SyntaxWarning: invalid escape sequence '\/'
<>:4: SyntaxWarning: invalid escape sequence '\/'
<>:3: SyntaxWarning: invalid escape sequence '\/'
<>:4: SyntaxWarning: invalid escape sequence '\/'
C:\Users\danis\AppData\Local\Temp\ipykernel_25548\2676205948.py:3: SyntaxWarning: invalid escape sequence '\/'
  regex_JS = '<script\b[^<]*(?:(?!<\/script>)<[^<]*)*<\/script>'
C:\Users\danis\AppData\Local\Temp\ipykernel_25548\2676205948.py:4: SyntaxWarning: invalid escape sequence '\/'
  regex_CSS = '<style\b[^<]*(?:(?!<\/style>)<[^<]*)*<\/style>'


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [7]:
import string

def remove_special_characters(text):
    # Remove special characthers
    my_puntuation = string.punctuation
    pattern_spc = re.compile('[%s]' % re.escape(string.punctuation))
    text_no_spe_charac = pattern_spc.sub(' ', text.lower())

    # Remove numbers
    text_no_numbers = re.sub(r'\d+', ' ', text_no_spe_charac)

    # Remove all single characters (e.g., 'a', 'b', 'c' that appear as standalone)
    text_no_single_charac = re.sub(r'\b\w\b', '', text_no_numbers)

    # Optional: clean up extra spaces left after removing single characters
    text_no_multiple_spaces = re.sub(r'\s+', ' ', text_no_single_charac).strip()

    # Remove prefixed 'b' (e.g., 'b' followed by any character)
    text_no_b = re.sub(r'^b\s', '', text_no_multiple_spaces)  # Replace 'b' followed by a character with that character

    # Convert everything to Lowercase
    text_lower = text_no_b.lower()

    return text_lower

X_train = X_train.apply(remove_special_characters)
X_test = X_test.apply(remove_special_characters)

X_train.head()




## Now let's work on removing stopwords
Remove the stopwords.

In [9]:
# Function to remove stopwords and stem words
def process_text(text):
    # Tokenize the text
    words = nltk.word_tokenize(text)
    
    # Remove stopwords
    filtered_words = [word for word in words if word.lower() not in stopwords.words("english")]
    
    return ' '.join(filtered_words)  # Join the words back into a single string

# Apply the function to the pandas Series
X_train_processed = X_train.apply(process_text)
X_test_processed = X_test.apply(process_text)

# Print the original and processed sentences
print("Original Sentences:")
print(X)
print("\nProcessed Sentences:")
print(X_train_processed)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [26]:
wordnet_lemma = nltk.stem.WordNetLemmatizer()
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.pos_tag(['Strip'])[0][1][0]
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN) # returns the word type (Noun if we have not found)

X_train_tokenized = X_train_processed.apply(nltk.tokenize.word_tokenize)
X_test_tokenized = X_test_processed.apply(nltk.tokenize.word_tokenize)




[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\danis\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# Trasform the documents using lemmatizer
X_train_lem =[]
for message in X_train_tokenized:
    message_final = []
    for word in message:
        # we can improve the results if we know if the word is a noun/vern/adverb or adjective
        message_final.append(wordnet_lemma.lemmatize(word, get_wordnet_pos(word))) # requires 'corpora/wordnet' -> nltk.download()
    X_train_lem.append(message_final)

X_test_lem =[]
for message in X_test_tokenized:
    message_final = []
    for word in message:
        # we can improve the results if we know if the word is a noun/vern/adverb or adjective
        message_final.append(wordnet_lemma.lemmatize(word, get_wordnet_pos(word))) # requires 'corpora/wordnet' -> nltk.download()
    X_test_lem.append(message_final)
    print()
    print('Before:',message)
    print('After: ', message_final)
    

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [11]:
spam = data_train[data_train['label'] == 1]['text']
spam_cleaned = spam.apply(remove_js_css_html_code)
spam_no_spc = spam_cleaned.apply(remove_special_characters)
spam_processed = spam_no_spc.apply(process_text)
spam_tokenized = spam_processed.apply(nltk.tokenize.word_tokenize)

spam_lem =[]
for message in spam_tokenized:
    message_final = []
    for word in message:
        # we can improve the results if we know if the word is a noun/vern/adverb or adjective
        message_final.append(wordnet_lemma.lemmatize(word, get_wordnet_pos(word))) # requires 'corpora/wordnet' -> nltk.download()
    spam_lem.append(message_final)

ham = data_train[data_train['label'] == 0]['text']

ham_cleaned = ham.apply(remove_js_css_html_code)
ham_no_spc = ham_cleaned.apply(remove_special_characters)
ham_processed = ham_no_spc.apply(process_text)
ham_tokenized = ham_processed.apply(nltk.tokenize.word_tokenize)

ham_lem =[]
for message in ham_tokenized:
    message_final = []
    for word in message:
        # we can improve the results if we know if the word is a noun/vern/adverb or adjective
        message_final.append(wordnet_lemma.lemmatize(word, get_wordnet_pos(word))) # requires 'corpora/wordnet' -> nltk.download()
    ham_lem.append(message_final)



In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# Convert processed_docs to strings
spam_as_strings = [' '.join(message_final) for message_final in spam_lem]
ham_as_strings = [' '.join(message_final) for message_final in ham_lem]

# Create the Bag of Words model
vectorizer = CountVectorizer()
X_spam = vectorizer.fit_transform(spam_as_strings)
words_spam = vectorizer.get_feature_names_out()

X_ham = vectorizer.fit_transform(ham_as_strings)
words_ham = vectorizer.get_feature_names_out()

# Show the Bag of Words feature names and the document-term matrix
print("Feature Names:", words_spam)
print("Document-Term Matrix:\n", X_spam.toarray())
print("Feature Names:", words_ham)
print("Document-Term Matrix:\n", X_ham.toarray())


Feature Names: ['aa' 'aaa' 'aabeiawaeaambiqaceqedeqh' ... 'â½trangers' 'â½x' 'â½ã']
Document-Term Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Feature Names: ['aaronovitchon' 'aback' 'abbas' ... 'zuma' 'zurich' 'âºwednesday']
Document-Term Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
import pandas as pd
# Sum the counts for each word across all documents
word_counts_spam = X_spam.toarray().sum(axis=0)
word_counts_ham = X_ham.toarray().sum(axis=0)

# Create a DataFrame to hold words and their corresponding counts
word_count_df_spam = pd.DataFrame({'word': words_spam, 'count': word_counts_spam})
word_count_df_ham = pd.DataFrame({'word': words_ham, 'count': word_counts_ham})

# Get the 10 most frequent words
most_frequent_words_spam = word_count_df_spam.nlargest(10, 'count')
most_frequent_words_ham = word_count_df_ham.nlargest(10, 'count')


# Print the results
display(most_frequent_words_spam)
display(most_frequent_words_ham)

,word,count
2920,br,1104
13491,money,987
222,account,899
2318,bank,802
7677,fund,786
20541,transfer,578
20528,transaction,555
3064,business,514
4315,country,513
13608,mr,490


,word,count
5051,state,169
4047,pm,127
828,call,115
4672,say,111
6022,work,107
6037,would,107
4172,president,99
5466,time,95
3453,mr,91
3659,obama,84


In [14]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['text'].str.contains(money_simbol_list, regex=False, na=False)*1
data_train['suspicious_words'] = data_train['text'].str.contains(suspicious_words, na=False)*1
data_train['text_len'] = data_train['text'].apply(lambda x: len(x)) 


data_test['money_mark'] = data_test['text'].str.contains(money_simbol_list, na=False)*1
data_test['suspicious_words'] = data_test['text'].str.contains(suspicious_words, na=False)*1
data_test['text_len'] = data_test['text'].apply(lambda x: len(x)) 



## How would work the Bag of Words with Count Vectorizer concept?

In [15]:
# the code above does not work

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

X_train_as_strings = [' '.join(message_final) for message_final in X_train_lem]
X_test_as_strings = [' '.join(message_final) for message_final in X_test_lem]

count_vectorizer = CountVectorizer (min_df = 1)#(min_df=0.1,max_df=0.7)
tfidf = TfidfTransformer(norm="l2")

term_freq_matrix_train = count_vectorizer.fit_transform(X_train_as_strings)
tfidf.fit(term_freq_matrix_train)
tf_idf_matrix_train = tfidf.transform(term_freq_matrix_train)

term_freq_matrix_test = count_vectorizer.fit_transform(X_test_as_strings)
tfidf.fit(term_freq_matrix_test)
tf_idf_matrix_test = tfidf.transform(term_freq_matrix_test)

print ("Vocabulary:", count_vectorizer.vocabulary_,'\n')
print(X_train_as_strings)
print()
print (tf_idf_matrix_train.todense())

import pandas as pd
# print idf values
df_idf = pd.DataFrame(tfidf.idf_, index=count_vectorizer.get_feature_names_out(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

Vocabulary: {'dear': 1208, 'sir': 3936, 'wish': 4673, 'go': 2036, 'offer': 3100, 'consider': 1046, 'partner': 3264, 'ei': 1524, 'mr': 2924, 'esamson': 1656, 'bamogu': 468, 'cfrom': 793, 'republic': 3669, 'sierra': 3912, 'leone': 2610, 'west': 4632, 'africa': 128, 'cnow': 912, 'seek': 3828, 'political': 3377, 'asylum': 392, 'senegal': 3844, 'unhcr': 4480, 'get': 2023, 'contact': 1057, 'search': 3813, 'business': 686, 'detail': 1294, 'emy': 1581, 'late': 2572, 'father': 1795, 'dr': 1415, 'emustapha': 1580, 'manage': 2731, 'director': 1335, 'rainbow': 3551, 'gold': 2040, 'diamond': 1316, 'mining': 2855, 'company': 965, 'ken': 2508, 'ema': 1550, 'ebut': 1472, 'kill': 2523, 'longing': 2670, 'civil': 870, 'war': 4594, 'property': 3497, 'totally': 4343, 'destroyed': 1292, 'ehowever': 1520, 'escape': 1657, 'precious': 3408, 'presently': 3427, 'deposit': 1264, 'safely': 3770, 'security': 3825, 'finance': 1851, 'dakar': 1180, 'saddle': 3766, 'problem': 3466, 'secure': 3824, 'trust': 4408, 'worth

,idf_weights
million,2.295972
state,2.314321
fund,2.352061
money,2.371479
account,2.391282
...,...
foreigncontract,5.610158
foreignremittance,5.610158
foremost,5.610158
fonctionnaires,5.610158


## And the Train a Classifier?

In [27]:
X = data_train['text']
X = X.apply(remove_js_css_html_code)
X = X.apply(remove_special_characters)

X = X.apply(process_text)

X = X.apply(nltk.tokenize.word_tokenize)

X_final =[]
for message in X:
    message_final = []
    for word in message:
        # we can improve the results if we know if the word is a noun/vern/adverb or adjective
        message_final.append(wordnet_lemma.lemmatize(word, get_wordnet_pos(word))) # requires 'corpora/wordnet' -> nltk.download()
    X_final.append(message_final)
print(X)

0      [dear, sir, strictly, private, business, propo...
1                                                     []
2      [nora, cheryl, emailed, dozens, memos, haiti, ...
3      [dear, sir, fmadam, know, proposal, might, sur...
4                                                  [fyi]
                             ...                        
995    [latest, sounds, contradictory, af, decide, sh...
996    [transfer, million, pounds, youraccount, name,...
997                 [barb, call, explain, back, country]
998    [yang, travelnot, free, tonite, may, work, tom...
999    [sbwhoeopsunday, february, pmhshaunh, talked, ...
Name: text, Length: 1000, dtype: object


In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
X =  [' '.join(message_final) for message_final in X_final]
y = data_train['label']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the test data
X_test_tfidf = vectorizer.transform(X_test)

# Initialize the Multinomial Naive Bayes classifier
model = MultinomialNB()

# Fit the model to the training data
model.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.91

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.84      0.92       116
           1       0.82      1.00      0.90        84

    accuracy                           0.91       200
   macro avg       0.91      0.92      0.91       200
weighted avg       0.93      0.91      0.91       200



((200, 4764), (800, 27336))

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [30]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
X =  [' '.join(message_final) for message_final in X_final]
y = data_train['label']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the test data
X_test_tfidf = vectorizer.transform(X_test)

# Initialize the Multinomial Naive Bayes classifier
model = MultinomialNB()

param_grid = {
    'alpha': [0.001, 0.01, 0.1, 0.5, 1.0],
    'fit_prior': [True, False]
}
# Set up the grid search
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Fit the model to the training data
grid_search.fit(X_train_tfidf, y_train)

# Best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)



Best Parameters: {'alpha': 0.001, 'fit_prior': True}
Best Accuracy: 0.96875


In [32]:
best_model = MultinomialNB(alpha=0.001, fit_prior=True)
best_model.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test_tfidf)


# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.965

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.94      0.97       116
           1       0.92      1.00      0.96        84

    accuracy                           0.96       200
   macro avg       0.96      0.97      0.96       200
weighted avg       0.97      0.96      0.97       200

